<a href="https://colab.research.google.com/github/sibot89/Data-Science/blob/main/MNIST_Classification_with_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/mnist/incomplete.EAHQ3S_3.0.1/mnist-train.tfrecord*...:   0%|          | 0…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/mnist/incomplete.EAHQ3S_3.0.1/mnist-test.tfrecord*...:   0%|          | 0/…

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [ ]:
mnist_info

tfds.core.DatasetInfo(
    name='mnist',
    full_name='mnist/3.0.1',
    description="""
    The MNIST database of handwritten digits.
    """,
    homepage='http://yann.lecun.com/exdb/mnist/',
    data_dir='/root/tensorflow_datasets/mnist/3.0.1',
    file_format=tfrecord,
    download_size=11.06 MiB,
    dataset_size=21.00 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    nondeterministic_order=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=60000, num_shards=1>,
    },
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann.lecun.com/exdb/mnist},
      volume={2},
      year

In [ ]:
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples

6000.0

In [ ]:
num_validation_samples = tf.cast(num_validation_samples, tf.int64)
num_validation_samples

<tf.Tensor: shape=(), dtype=int64, numpy=6000>

In [ ]:
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples

10000

In [ ]:
num_test_samples = tf.cast(num_test_samples, tf.int64)
num_test_samples

<tf.Tensor: shape=(), dtype=int64, numpy=10000>

In [ ]:
def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255.
  return image, label

In [ ]:
scaled_train_data = mnist_train.map(scale)
scaled_tast_data = mnist_test.map(scale)

In [ ]:
BUFFER_SIZE = 10000
shuffled_train_data = scaled_train_data.shuffle(BUFFER_SIZE)
shuffled_train_data

<_ShuffleDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [ ]:
validation_data = shuffled_train_data.take(num_validation_samples)
train_data = shuffled_train_data.skip(num_validation_samples)

In [ ]:
BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)

validation_data = validation_data.batch(num_validation_samples)

test_data = scaled_tast_data.batch(num_test_samples)

In [ ]:
validation_inputs, validation_targets = next(iter(validation_data))

Outlining the Model

In [ ]:
input_size = 784
output_size = 10
hidden_layer_size = 50

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
NUM_EPOCHS = 5

model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs,validation_targets), verbose=2)

Epoch 1/5
540/540 - 10s - 18ms/step - accuracy: 0.8844 - loss: 0.4122 - val_accuracy: 0.9325 - val_loss: 0.2143
Epoch 2/5
540/540 - 4s - 7ms/step - accuracy: 0.9463 - loss: 0.1810 - val_accuracy: 0.9493 - val_loss: 0.1615
Epoch 3/5
540/540 - 4s - 7ms/step - accuracy: 0.9601 - loss: 0.1373 - val_accuracy: 0.9595 - val_loss: 0.1246
Epoch 4/5
540/540 - 6s - 11ms/step - accuracy: 0.9666 - loss: 0.1114 - val_accuracy: 0.9682 - val_loss: 0.1052
Epoch 5/5
540/540 - 4s - 7ms/step - accuracy: 0.9715 - loss: 0.0940 - val_accuracy: 0.9727 - val_loss: 0.0938


In [ ]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 882ms/step - accuracy: 0.9685 - loss: 0.1045


In [ ]:
print('Test loss of our model is: {0:.3f}. And our test accuracy is: {1:.3f}%'.format(test_loss, test_accuracy))

Test loss of our model is: 0.105. And our test accuracy is: 0.969%
